# Контекстные заголовки чанков (CCH) в Simple RAG

Retrieval-Augmented Generation (RAG) повышает фактическую точность языковых моделей, извлекая релевантные внешние знания перед генерацией ответа. Однако стандартное разбиение на чанки часто теряет важный контекст, снижая эффективность поиска.

Contextual Chunk Headers (CCH) улучшают RAG, добавляя высокоуровневый контекст (например, заголовки документов или разделов) к каждому чанку перед их векторизацией. Это повышает качество поиска и предотвращает ответы вне контекста.

## Шаги в этом ноутбуке:

1. **Загрузка данных**: Загрузка и предварительная обработка текстовых данных.
2. **Разбиение с контекстными заголовками**: Извлечение заголовков разделов и добавление их к чанкам.
3. **Создание эмбеддингов**: Преобразование чанков с контекстом в числовые представления.
4. **Семантический поиск**: Поиск релевантных чанков по пользовательскому запросу.
5. **Генерация ответа**: Использование языковой модели для генерации ответа из найденного текста.
6. **Оценка**: Анализ точности ответов с помощью системы оценки.

## Настройка окружения
Начнём с импорта необходимых библиотек.

In [1]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz
from tqdm import tqdm

## Извлечение текста и определение заголовков разделов
Извлекаем текст из PDF, одновременно определяя заголовки разделов (потенциальные заголовки для чанков).

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    Извлекает текст из PDF файла и выводит первые `num_chars` символов.

    Аргументы:
    pdf_path (str): Путь к PDF файлу.

    Возвращает:
    str: Извлечённый текст из PDF.
    """
    # Открываем PDF файл
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Инициализируем пустую строку для хранения текста

    # Итерируемся по страницам PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Получаем страницу
        text = page.get_text("text")  # Извлекаем текст со страницы
        all_text += text  # Добавляем текст к общей строке

    return all_text  # Возвращаем извлечённый текст

## Настройка клиента OpenAI API
Инициализируем клиент OpenAI для генерации эмбеддингов и ответов.

In [ ]:
# Инициализируем клиент OpenAI с базовым URL и API ключом
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # Получаем API ключ из переменных окружения
)

## Разбиение текста с контекстными заголовками
Для улучшения поиска генерируем описательные заголовки для каждого чанка с помощью LLM модели.

In [4]:
def generate_chunk_header(chunk, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Генерирует заголовок для заданного текстового чанка с помощью LLM.

    Аргументы:
    chunk (str): Текстовый чанк для генерации заголовка.
    model (str): Модель для генерации заголовка. По умолчанию "meta-llama/Llama-3.2-3B-Instruct".

    Возвращает:
    str: Сгенерированный заголовок.
    """
    # Определяем системный промт для управления поведением ИИ
    system_prompt = "Generate a concise and informative title for the given text."
    
    # Генерируем ответ от ИИ модели на основе системного промта и текстового чанка
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # Возвращаем сгенерированный заголовок, удаляя лишние пробелы
    return response.choices[0].message.content.strip()